In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [4]:
BATCH_SIZE = 64
EPOCHS = 3

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))])

In [5]:
trainset = torchvision.datasets.MNIST(root = './data',
                                                        train = True,
                                                        download = True,
                                                        transform = transform)

trainloader = torch.utils.data.DataLoader(trainset,
                                                             batch_size = BATCH_SIZE,
                                                             shuffle = True,
                                                             num_workers = 2)

0.00B [00:00, ?B/s]

 99%|█████████▉| 9.84M/9.91M [00:17<00:00, 590kB/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz



0.00B [00:00, ?B/s]
  0%|          | 0.00/28.9k [00:00<?, ?B/s]


32.8kB [00:00, 138kB/s]                    
0.00B [00:00, ?B/s]
  0%|          | 0.00/1.65M [00:00<?, ?B/s]

...


  2%|▏         | 32.8k/1.65M [00:00<00:05, 315kB/s]
  3%|▎         | 49.2k/1.65M [00:00<00:09, 164kB/s]
  7%|▋         | 115k/1.65M [00:00<00:08, 180kB/s] 
 12%|█▏        | 197k/1.65M [00:01<00:07, 201kB/s]
 18%|█▊        | 295k/1.65M [00:01<00:05, 247kB/s]
 23%|██▎       | 377k/1.65M [00:01<00:04, 289kB/s]
 28%|██▊       | 467k/1.65M [00:01<00:03, 333kB/s]
 34%|███▍      | 557k/1.65M [00:01<00:02, 370kB/s]
 40%|███▉      | 655k/1.65M [00:01<00:02, 407kB/s]
 45%|████▍     | 737k/1.65M [00:02<00:02, 434kB/s]
 51%|█████     | 836k/1.65M [00:02<00:01, 459kB/s]
 57%|█████▋    | 934k/1.65M [00:02<00:01, 482kB/s]
 63%|██████▎   | 1.04M/1.65M [00:02<00:01, 508kB/s]
 70%|██████▉   | 1.15M/1.65M [00:02<00:00, 528kB/s]
 76%|███████▌  | 1.25M/1.65M [00:03<00:00, 529kB/s]
 79%|███████▉  | 1.30M/1.65M [00:03<00:00, 477kB/s]
 87%|████████▋ | 1.44M/1.65M [00:03<00:00, 538kB/s]
 91%|█████████▏| 1.51M/1.65M [00:03<00:00, 446kB/s]
 99%|█████████▉| 1.64M/1.65M [00:03<00:00, 518kB/s]

0.00B [00:00, ?B/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz




  0%|          | 0.00/4.54k [00:00<?, ?B/s]

8.19kB [00:00, 18.4kB/s]                   

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


9.92MB [00:30, 590kB/s]                            
1.65MB [00:22, 518kB/s]                            

In [7]:
testset = torchvision.datasets.MNIST(root = './data',
                                                      train = False,
                                                       download = True,
                                                        transform = transform)

testloader = torch.utils.data.DataLoader(testset, 
                                                            batch_size = BATCH_SIZE,
                                                            shuffle = False,
                                                            num_workers = 2)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, 
                                            out_channels = 32, 
                                            kernel_size = 3,
                                            stride = 1) 
        
        self.conv2 = nn.Conv2d(in_channels = 32,
                                            out_channels = 64,
                                            kernel_size = 3,
                                            stride = 1) 
        
        self.pool = nn.MaxPool2d(kernel_size = 2, 
                                               stride = 2) 
        
        self.dropout1 = nn.Dropout2d(p = 0.5)
        self.fc1 = nn.Linear(in_features = 12 * 12 * 64, out_features = 128)
        self.dropout2 = nn.Dropout2d(p = 0.5)
        self.fc2 = nn.Linear(128, 10) # There are 10 classes [0,1,2,3,4,5,6,7,8,9]
        
    
    def forward(self, x):
        x = F.relu(self.conv1(x)) #(28, 28, 1) -> (26, 26, 32)
        x = self.pool(F.relu(self.conv2(x))) #(26, 26, 32) -> (24, 24, 64) -> (12, 12, 64)
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64) #reshape from (12, 12, 64) to (1, 9216)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [11]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

# Train

In [17]:
for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        optimizer.zero_grad()
        
        #forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i +1, running_loss / 100))
            running_loss = 0.0
print("Finished Training")

[1,   100] loss: 2.198
[1,   200] loss: 2.159
[1,   300] loss: 2.110
[1,   400] loss: 2.034
[1,   500] loss: 1.958
[1,   600] loss: 1.843
[1,   700] loss: 1.722
[1,   800] loss: 1.586
[1,   900] loss: 1.426
[2,   100] loss: 1.249
[2,   200] loss: 1.138
[2,   300] loss: 1.031
[2,   400] loss: 0.966
[2,   500] loss: 0.905
[2,   600] loss: 0.846
[2,   700] loss: 0.794
[2,   800] loss: 0.780
[2,   900] loss: 0.738
[3,   100] loss: 0.680
[3,   200] loss: 0.671
[3,   300] loss: 0.679
[3,   400] loss: 0.644
[3,   500] loss: 0.638
[3,   600] loss: 0.628
[3,   700] loss: 0.618
[3,   800] loss: 0.604
[3,   900] loss: 0.552
Finished Training


In [18]:
correct = 0
total = 0

with torch.no_grad():
    for (images, labels) in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

Accuracy of the network: 84 %
